In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns
from datetime import datetime
# close the warning messages
import warnings
warnings.filterwarnings("ignore")

In [41]:
def calculate_age(birth_date_str):
    birth_date = datetime.strptime(birth_date_str, '%Y-%m-%d')
    current_date = datetime.now()
    age = current_date.year - birth_date.year
    if (current_date.month, current_date.day) < (birth_date.month, birth_date.day):
        age -= 1
    return age

In [42]:
timeLocal = time.localtime(time.time())
exp1File = f"无期望选择AI+教师_{timeLocal.tm_mon}_{timeLocal.tm_mday}_{timeLocal.tm_hour}h{timeLocal.tm_min}.xlsx"
exp2File = f"有期望选择3选项_{timeLocal.tm_mon}_{timeLocal.tm_mday}_{timeLocal.tm_hour}h{timeLocal.tm_min}.xlsx"

dataFrame = {}
fileList = ['三选项2元','三选项5元','三选项免费']
for i, rawData in enumerate(fileList):
    dfName = f'df{i+1}'
    dataFrame[dfName] = pd.read_excel(f'{rawData}.xlsx')

df1 = dataFrame['df1']
df2 = dataFrame['df2']
df3 = dataFrame['df3']

dfMerge = pd.concat([df1,df2,df3], ignore_index=True)


dfMerge['expIndex'] = [0] * len(dfMerge)

for i in range(len(dfMerge)):
    if "意愿题" in dfMerge["随机元素"][i]:
        dfMerge["expIndex"][i] = 2
    else:
        dfMerge["expIndex"][i] = 1
# rawData中expIndex为1的提取为exp1结构体，expIndex为2的提取为exp2结构体
exp1 = dfMerge[dfMerge['expIndex'] == 1]
exp2 = dfMerge[dfMerge['expIndex'] == 2]

exp2.index = range(len(exp2))
exp1.index = range(len(exp1))

In [43]:
exp2

,作答ID,用户ID,开始时间,结束时间,作答时长(秒),作答渠道,发布ID,问卷发布名称,IP,经度,...,您对评分结果比较满意的原因是？.1,您认为该评分系统不公平的主要原因是？.1,您认为该评分系统公平的主要原因是？.1,您认为评分结果不公平的主要原因是？.5,您认为评分结果公平的主要原因是？.5,您的性别为,请选择您的生日,您的职业,在中译英过程中，您是否使用了任何辅助工具？,expIndex
0,XyvZ0RBrpnr,QL002n0L,2024-09-18 19:30:17,2024-09-18 19:39:40,563,链接二维码,EVfEJr,心理评测,117.136.111.155,120.155150,...,NaN,NaN,NaN,NaN,NaN,女,2001-10-09,在读学生,我没有使用辅助工具,2
1,NP4xjrQAn9q,z7lGDX2M,2024-09-18 11:47:54,2024-09-18 11:56:25,511,链接二维码,EVfEJr,心理评测,36.21.214.37,119.922930,...,NaN,NaN,NaN,NaN,NaN,女,2005-06-27,在读学生,我没有使用辅助工具,2
2,mNa6b5blzDz,pLzq3Q87,2024-09-17 17:07:17,2024-09-17 17:15:19,482,链接二维码,EVfEJr,心理评测,124.160.39.134,120.298501,...,NaN,NaN,NaN,NaN,NaN,女,2000-10-13,在读学生,我没有使用辅助工具,2
3,g92RbrrWpbn,leNXdgW7,2024-09-17 17:05:14,2024-09-17 17:13:18,484,链接二维码,EVfEJr,心理评测,223.104.160.108,120.155150,...,NaN,NaN,NaN,NaN,NaN,男,2003-10-17,在读学生,我没有使用辅助工具,2
4,Q1RpaRqpV2W,qM6zg4vM,2024-09-17 16:38:46,2024-09-17 16:45:15,389,链接二维码,EVfEJr,心理评测,210.32.43.223,120.298501,...,NaN,NaN,NaN,NaN,NaN,女,2004-08-31,在读学生,我没有使用辅助工具,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,nZEXp2mrP3B,AM9DDl4e,2024-09-09 16:04:55,2024-09-09 16:12:58,483,链接二维码,aERzIf,心理评测V1,39.174.135.102,120.130260,...,NaN,NaN,NaN,NaN,NaN,女,2005-01-09,在读学生,我没有使用辅助工具,2
275,g9xAaoAEzow,5LwmxXqM,2024-09-09 16:04:05,2024-09-09 16:11:56,471,链接二维码,aERzIf,心理评测V1,39.174.132.130,120.130260,...,NaN,NaN,NaN,NaN,NaN,男,2005-02-19,在读学生,我没有使用辅助工具,2
276,nZEXpRWV68N,ve8yp9BM,2024-09-09 16:02:57,2024-09-09 16:11:37,520,链接二维码,aERzIf,心理评测V1,210.32.3.78,120.130260,...,NaN,NaN,NaN,NaN,NaN,女,2004-11-09,在读学生,我没有使用辅助工具,2
277,Q16rzNrPOVb,kM11g9QM,2024-09-09 16:03:26,2024-09-09 16:11:17,471,链接二维码,aERzIf,心理评测V1,39.174.132.87,120.130260,...,NaN,NaN,NaN,NaN,NaN,女,2005-05-23,在读学生,我没有使用辅助工具,2


In [44]:
# exp1新增gender(M/F)，Stu/Work，AE，score(1)/evaluator(2)，AS，H/M/L，AS-IS，Ss,SF,Ps,Pf列，全部赋空值
exp1.dropna(axis=1, how='all', inplace=True)
exp1.rename(columns={'您认为您的答案可以得到几分？': 'IS'}, inplace=True)
exp1['gender(M/F)'] = ''
exp1['Stu/Work'] = ''
exp1['AE'] = ''
exp1['score(1)/evaluator(2)'] = ''
exp1['I_H/M/L'] = ''
exp1['AS'] = ''
exp1['A_H/M/L'] = ''
exp1['AS-IS'] = ''
exp1['SS'] = ''
exp1['SF'] = ''
exp1['ES'] = ''
exp1['EF'] = ''

exp1.rename(columns={'作答时长(秒)':'timeDuration'},inplace=True)

exp1['score(1)/evaluator(2)'] = exp1['随机元素'].apply(lambda x: 1 if '感知 1' in x else 2)

exp1['AE'] = 3

# 将列名中包含“满分10分”的列相加到“AS”这一列
exp1['AS'] = exp1.filter(regex='满分10分').astype(float).sum(axis=1)

# 将列名中包含“评分结果是否公平”的列相加到“SF”这一列
exp1['SF'] = exp1.filter(regex='评分结果是否公平').astype(float).sum(axis=1)
# 将列名中包含"评分结果是否满意"的列相加到"SS"这一列
exp1['SS'] = exp1.filter(regex='评分结果是否满意').astype(float).sum(axis=1)

# 将列名中包含“评分结果是否客观”的列相加到“Ss”这一列
exp1['SS'] = exp1.filter(regex='评分结果是否满意').astype(float).sum(axis=1)

# 将列名中包含“系统是否满意”或者“教师是否满意”的列相加到“ES”这一列
exp1['ES'] = exp1.filter(regex='系统是否满意|教师是否满意').astype(float).sum(axis=1)

# 将列名中包含“系统是否满意”或者“教师是否满意”的列相加到“ES”这一列
exp1['EF'] = exp1.filter(regex='系统是否公平|教师是否公平').astype(float).sum(axis=1)

# AS-IS
exp1['AS-IS'] = exp1['AS'] - exp1['IS']

# 如果“您的性别”一列为“男”，则gender(M/F)一列为1，否则为2
exp1['gender(M/F)'] = exp1['您的性别为'].apply(lambda x: 1 if x == '男' else 2)

# 如果“您的职业”一列为“学生”，则Stu/Work一列为1，否则为2
exp1['Stu/Work'] = exp1['您的职业'].apply(lambda x: 1 if x == '在读学生' else 2)

# 如果AS小于4，H/M/L一列为1，如果AS小于7，H/M/L一列为2，否则为3
exp1['A_H/M/L'] = exp1['AS'].apply(lambda x: 1 if x < 4 else 2 if x < 8 else 3)

exp1['I_H/M/L'] = exp1['IS'].apply(lambda x: 1 if x < 4 else 2 if x < 8 else 3)

exp1['age'] = exp1['请选择您的生日'].apply(calculate_age)

# exp1 = exp1[exp1['AS-IS']<7]

columns1 = ['IS','gender(M/F)','Stu/Work','AE','score(1)/evaluator(2)','I_H/M/L','AS','A_H/M/L','AS-IS','SS','SF','ES','EF','age','timeDuration']

exp1 = exp1[columns1]

exp1.to_excel(exp1File, index=False)


In [45]:
exp2.dropna(axis=1, how='all', inplace=True)
exp2.rename(columns={'您认为您的答案可以得到几分？': 'IS'}, inplace=True)
exp2['gender(M/F)'] = ''
exp2['Stu/Work'] = ''
exp2['IE'] = ''
exp2['AE'] = ''
exp2['congruency'] = ''
exp2['AS'] = ''
exp2['score(1)/evaluator(2)'] = ''
exp2['I_H/M/L'] = ''
exp2['AS'] = ''
exp2['A_H/M/L'] = ''
exp2['AS-IS'] = ''
exp2['SS'] = ''
exp2['SF'] = ''
exp2['ES'] = ''
exp2['EF'] = ''
exp2['age'] = ''
exp2.rename(columns={'作答时长(秒)':'timeDuration'},inplace=True)
exp2['IE'] = exp2['你更希望谁来给你的答案打分？（经过多轮测试，该AI英语写作评分系统和大学英语老师水平接近）'].apply(lambda x: 1 if x =='AI英语写作评分系统' else 2 if x == '大学英语教师' else 3)

# 给各列赋值
for i in range(len(exp2)):
    if "感知 1" in exp2['随机元素'][i]:
        exp2['score(1)/evaluator(2)'][i] = 1
    elif "感知 2" in exp2['随机元素'][i]:
        exp2['score(1)/evaluator(2)'][i] = 2

for i in range(len(exp2)):
    if "AI评分" in exp2['随机元素'][i]:
        exp2['AE'][i] = 1
    elif "AI+教师" in exp2['随机元素'][i]:
        exp2['AE'][i] = 3
    else:
        exp2['AE'][i] = 2


# 将列名中包含“系统是否满意”或者“教师是否满意”的列相加到“ES”这一列
exp2['EF'] = exp2.filter(regex='系统是否公平|教师是否公平').astype(float).sum(axis=1)

# AS-IS
exp1['AS-IS'] = exp1['AS'] - exp1['IS']

# 将列名中包含“评分结果是否公平”的列相加到“SF”这一列
exp2['SF'] = exp2.filter(regex='评分结果是否公平').astype(float).sum(axis=1)

# 将列名中包含“评分结果是否客观”的列相加到“Ss”这一列
exp2['SS'] = exp2.filter(regex='评分结果是否满意').astype(float).sum(axis=1)

exp2
# 将列名中包含“系统是否满意”或者“教师是否满意”的列相加到“ES”这一列
exp2['ES'] = exp2.filter(regex='系统是否满意|教师是否满意').astype(float).sum(axis=1)

# 将列名中包含“系统是否满意”或者“教师是否满意”的列相加到“ES”这一列
exp2['EF'] = exp2.filter(regex='系统是否公平|教师是否公平').astype(float).sum(axis=1)

# AS-IS
exp2['AS-IS'] = exp2['AS'] - exp2['IS']

# 如果“您的性别”一列为“男”，则gender(M/F)一列为1，否则为2
exp2['gender(M/F)'] = exp2['您的性别为'].apply(lambda x: 1 if x == '男' else 2)

# 如果“您的职业”一列为“学生”，则Stu/Work一列为1，否则为2
exp2['Stu/Work'] = exp2['您的职业'].apply(lambda x: 1 if x == '在读学生' else 2)

# 如果AS小于4，H/M/L一列为1，如果AS小于7，H/M/L一列为2，否则为3
exp2['A_H/M/L'] = exp2['AS'].apply(lambda x: 1 if x < 4 else 2 if x < 8 else 3)

exp2['I_H/M/L'] = exp2['IS'].apply(lambda x: 1 if x < 4 else 2 if x < 8 else 3)

# 得到年龄
exp2['age'] = exp2['请选择您的生日'].apply(calculate_age)

columns2 = ['IS','gender(M/F)','Stu/Work','IE','AE','congruency','score(1)/evaluator(2)','I_H/M/L','AS','A_H/M/L','AS-IS','SS','SF','ES','EF','age','timeDuration']

exp2 = exp2[columns2]

exp2.to_excel(exp2File, index=False)


In [46]:
import site
print(site.getsitepackages())
import pandas as pd
pd.read_csv()


['c:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python38', 'c:\\Users\\Lenovo\\AppData\\Local\\Programs\\Python\\Python38\\lib\\site-packages']
